In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os
# Define paths to your dataset
train_dir = r'C:\Users\thota\OneDrive\Desktop\uvk Application\cervical fracture\train'
test_dir = r'C:\Users\thota\OneDrive\Desktop\uvk Application\cervical fracture\val'
# Image parameters
img_height, img_width = 256, 256
batch_size = 32
# More aggressive data augmentation for the training set
train_datagen = ImageDataGenerator(
rescale=1./255,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
rotation_range=40, # Introduce rotation
width_shift_range=0.2,
height_shift_range=0.2,
fill_mode='nearest'
)
# Rescale the test set (no augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)
# Load training data
train_generator = train_datagen.flow_from_directory(
train_dir,
target_size=(img_height, img_width),
batch_size=batch_size,
class_mode='binary' # You might need to adjust this based on your dataset structure
)
# Load test data
test_generator = test_datagen.flow_from_directory(
test_dir,
target_size=(img_height, img_width),
batch_size=batch_size,
class_mode='binary' # You might need to adjust this based on your dataset structure
)
# Build a more complex CNN model
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))
# Compile the model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
loss='binary_crossentropy',
metrics=['accuracy'])
# Train the model
history=model.fit(
train_generator,
steps_per_epoch=train_generator.samples // batch_size,
epochs=5, # Increase the number of epochs
validation_data=test_generator,
validation_steps=test_generator.samples // batch_size
)
# Save the model
model.save('spinal_cord_model_updated.h5')

C:\Users\thota\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"



Found 3800 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


Epoch 1/5


118/118 [==============================] - 364s 3s/step - loss: 0.5882 - accuracy: 0.6762 - val_loss: 0.5969 - val_accuracy: 0.5182
Epoch 2/5
118/118 [==============================] - 356s 3s/step - loss: 0.4595 - accuracy: 0.7826 - val_loss: 0.6992 - val_accuracy: 0.5156
Epoch 3/5
118/118 [==============================] - 358s 3s/step - loss: 0.3870 - accuracy: 0.8384 - val_loss: 0.5110 - val_accuracy: 0.8021
Epoch 4/5
118/118 [==============================] - 356s 3s/step - loss: 0.3343 - accuracy: 0.8601 - val_loss: 0.6384 - val_accuracy: 0.6693
Epoch 5/5
118/118 [==============================] - 356s 3s/step - loss: 0.3087 - accuracy: 0.8689 - val_loss: 0.4348 - val_accuracy: 0.8620


C:\Users\thota\anaconda3\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [21]:
from tensorflow.keras.models import load_model
mymodel = load_model('spinal_cord_model_updated.h5')

In [3]:
pip install PyQt6

     ---------------------------------------- 6.5/6.5 MB 97.9 kB/s eta 0:00:00
     -------------------------------------- 73.0/73.0 kB 502.3 kB/s eta 0:00:00
     ---------------------------------------- 62.4/62.4 MB 1.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


# App

In [2]:
import sys
from PyQt6.QtWidgets import QApplication, QMainWindow, QLabel, QPushButton, QFileDialog
from PyQt6.QtGui import QPixmap
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
class SmartPostureApp(QMainWindow):
    def  _init_(self):
        super()._init_()
# Load the trained model
        self.model_path=r"C:\Users\thota\OneDrive\Desktop\uvk Application\spinal_cord_model_updated.h5"
        self.model = load_model(self.model_path)
# Initialize UI
        self.initUI()
    def initUI(self):
        self.setWindowTitle("smart posture Prediction")
        self.setGeometry(100, 100, 600, 400)
# Create QLabel for displaying the selected image
        self.image_label = QLabel(self)
        self.image_label.setGeometry(50, 50, 200, 200)
        # Create QLabel for displaying the prediction result
        self.result_label = QLabel(self)
        self.result_label.setGeometry(50, 270, 500, 30)
# Create QPushButton for inserting an image
        self.insert_button = QPushButton("Insert Image", self)
        self.insert_button.setGeometry(300, 50, 200, 50)
        self.insert_button.clicked.connect(self.insert_image)
    def preprocess_image(self, img_path, target_size=(128, 128)):
        img = image.load_img(img_path, target_size=target_size)
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array /= 255.0 # Normalize pixel values
        return img_array
    def predict_posture(self, img_path):
        img_array = self.preprocess_image(img_path)
        prediction = self.model.predict(img_array)
        return "need doctor consultation" if prediction > 0.5 else "Not required doctor"
    def insert_image(self):
        file_dialog = QFileDialog()
        file_path, _ = file_dialog.getOpenFileName(self, "Insert Image", "", "Image Files (*.png *.jpg*.jpeg)")
        if file_path:
            pixmap = QPixmap(file_path)
            pixmap = pixmap.scaled(200, 200)
            self.image_label.setPixmap(pixmap)
            result = self.predict(file_path)
            self.result_label.setText(f"The X-Ray is predicted to be: {result}")
if __name__ == "__main__":
    app = QApplication(sys.argv)
    mainWin = SmartPostureApp()
    mainWin.show()
    sys.exit(app.exec())

ModuleNotFoundError: No module named 'PyQt6'